In [39]:
from PIL import Image
import yaml
import requests
from transformers import AutoProcessor,  AutoTokenizer # LlavaForConditionalGeneration,
from omegaconf import OmegaConf as om
from tqdm import tqdm 
from typing import Any, Dict, List, Optional, Union
import copy
import re
import torch
import streaming
from omegaconf import DictConfig, ListConfig

# import sys
# sys.path.append("/mnt/workdisk/jasmine/llm-foundry")
# from scripts.train.train import validate_config, build_composer_model
# from llmfoundry.data.dataloader import build_dataloader
# from llmfoundry.utils.config_utils import (log_config, pop_config,
#                                            process_init_device,
#                                            update_batch_size_info)
# from llmfoundry.utils.builders import (add_metrics_to_eval_loaders,
#                                        build_algorithm, build_callback,
#                                        build_evaluators, build_logger,
#                                        build_optimizer, build_scheduler,
#                                        build_tokenizer)

import json
import os

In [1]:
! pip install transformers==4.38.2

In [3]:
tokenizer = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')
chat_template =  "{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif USE_DEFAULT_PROMPT == true and not 'system' in messages[0]['role'] %}{% set loop_messages = messages %}{% set system_message = 'DEFAULT_SYSTEM_PROMPT' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 %}{% if system_message != false %}{{ '<|im_start|>system\n' + system_message.strip() + '<|im_end|>\n'}}{% endif %}{{ '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' }}{% else %}{{ '\n' + '<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' }}{% endif %}{% if (add_generation_prompt == true and loop.last) %}{{ '\n' + '<|im_start|>' + 'assistant' + '\n' }}{% endif %}{% endfor %}"
tokenizer.chat_template = chat_template
tokenizer.add_tokens(['<image>', '<pad>', '<|im_end|>', '<|im_start|>'], special_tokens=True)
print(tokenizer)

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32003: AddedToken("<|im_start

In [4]:
example = {'messages': [{'role': 'system', 'content': 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.'}, {'role': 'user', 'content': '<image>\nDo you see any buses or cars?\nAnswer the question using a single word or phrase.'}, {'role': 'assistant', 'content': 'Yes'}, {'role': 'user', 'content': 'Is there any fence in this photograph?'}, {'role': 'assistant', 'content': 'Yes'}, {'role': 'user', 'content': 'Do you see a bus that is not red?'}, {'role': 'assistant', 'content': 'No'}, {'role': 'user', 'content': 'Which color is that bus?'}, {'role': 'assistant', 'content': 'Red'}, {'role': 'user', 'content': 'What is the sidewalk made of?'}, {'role': 'assistant', 'content': 'Concrete'}]}
messages = example['messages']
print(messages)

[{'role': 'system', 'content': 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.'}, {'role': 'user', 'content': '<image>\nDo you see any buses or cars?\nAnswer the question using a single word or phrase.'}, {'role': 'assistant', 'content': 'Yes'}, {'role': 'user', 'content': 'Is there any fence in this photograph?'}, {'role': 'assistant', 'content': 'Yes'}, {'role': 'user', 'content': 'Do you see a bus that is not red?'}, {'role': 'assistant', 'content': 'No'}, {'role': 'user', 'content': 'Which color is that bus?'}, {'role': 'assistant', 'content': 'Red'}, {'role': 'user', 'content': 'What is the sidewalk made of?'}, {'role': 'assistant', 'content': 'Concrete'}]


In [54]:
full_conversation = tokenizer.apply_chat_template(messages, tokenize=False)
tokenized_conversation = tokenizer(full_conversation)



input_id = torch.tensor(tokenized_conversation['input_ids'])


In [33]:
print(full_conversation)
print(input_id)

<|im_start|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|>
<|im_start|>user
<image>
Do you see any buses or cars?
Answer the question using a single word or phrase.<|im_end|>
<|im_start|>assistant
Yes<|im_end|>
<|im_start|>user
Is there any fence in this photograph?<|im_end|>
<|im_start|>assistant
Yes<|im_end|>
<|im_start|>user
Do you see a bus that is not red?<|im_end|>
<|im_start|>assistant
No<|im_end|>
<|im_start|>user
Which color is that bus?<|im_end|>
<|im_start|>assistant
Red<|im_end|>
<|im_start|>user
What is the sidewalk made of?<|im_end|>
<|im_start|>assistant
Concrete<|im_end|>
tensor([    1, 32003,  1587,    13,  1976,   460,   264, 10865, 28725,  3116,
         1007,   304,  6858, 13892, 28723, 17484,  4372,   390,  1316,  3071,
          390,  2572, 28723, 32002, 28705,    13, 32003,  2188,    13, 32000,
        28705,    13,  4957,   368,  1032,   707,  1579,   274,   442,  8300,
        28804,    13,  2820, 16

In [48]:
def tokenizer_image_token(prompt, tokenizer, image_token_index=IMAGE_TOKEN_INDEX, return_tensors=None):
    prompt_chunks = [tokenizer(chunk).input_ids for chunk in prompt.split('<image>')]

    def insert_separator(X, sep):
        return [ele for sublist in zip(X, [sep]*len(X)) for ele in sublist][:-1]

    input_ids = []
    offset = 0
    if len(prompt_chunks) > 0 and len(prompt_chunks[0]) > 0 and prompt_chunks[0][0] == tokenizer.bos_token_id:
        offset = 1
        input_ids.append(prompt_chunks[0][0])

    for x in insert_separator(prompt_chunks, [image_token_index] * (offset + 1)):
        input_ids.extend(x[offset:])

    if return_tensors is not None:
        if return_tensors == 'pt':
            return torch.tensor(input_ids, dtype=torch.long)
        raise ValueError(f'Unsupported tensor type: {return_tensors}')
    return input_ids

In [55]:
# trying to split based off of pre-tokenized output

IMAGE_TOKEN_INDEX = 32000 # i think
_HF_IGNORE_INDEX = -100
sep = '<|im_end|>\n' 
start = '<|im_start|>'
sep_n_tokens = 2 # NOTE: check if '<|im_end|>\n' splits into two tokens bc will need that for later

rounds = full_conversation.split(sep) 
cur_len = 0 #  1 ??
# target[:cur_len] = _HF_IGNORE_INDEX

target = input_id.clone() # copy.deepcopy(input_id)
print('targets before', target)
for i, rou in enumerate(rounds):
    role = rou.split('\n')[0][len(start):] # split on first newline then grab everything after <|im_start|>
    # print(role)
    # print(rou)

    round_len = len(tokenizer_image_token(rou, tokenizer))
    if role == 'assistant':
        # need to separate <|im_start|>assistant\n from the rest of stuff
        pass
        # round_len
        # instruction_len = 0
    else: # system or user
        
        target[cur_len: cur_len + round_len] = _HF_IGNORE_INDEX
        # print(round_len)

    cur_len += round_len
    print('cur len', cur_len)

    print('--')
    # break
print('targets after', target)

targets before tensor([    1, 32003,  1587,    13,  1976,   460,   264, 10865, 28725,  3116,
         1007,   304,  6858, 13892, 28723, 17484,  4372,   390,  1316,  3071,
          390,  2572, 28723, 32002, 28705,    13, 32003,  2188,    13, 32000,
        28705,    13,  4957,   368,  1032,   707,  1579,   274,   442,  8300,
        28804,    13,  2820, 16981,   272,  2996,  1413,   264,  2692,  1707,
          442, 14804, 28723, 32002, 28705,    13, 32003, 13892,    13,  5613,
        32002, 28705,    13, 32003,  2188,    13,  2301,   736,   707, 17210,
          297,   456,  9180, 28804, 32002, 28705,    13, 32003, 13892,    13,
         5613, 32002, 28705,    13, 32003,  2188,    13,  4957,   368,  1032,
          264,  1579,   369,   349,   459,  2760, 28804, 32002, 28705,    13,
        32003, 13892,    13,  2501, 32002, 28705,    13, 32003,  2188,    13,
        26703,  3181,   349,   369,  1579, 28804, 32002, 28705,    13, 32003,
        13892,    13,  7516, 32002, 28705,    13,

In [62]:
tokenizer

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32003: AddedToken("<|im_start

In [65]:
print(tokenizer.convert_tokens_to_ids('<|im_start|>'))
print(tokenizer.convert_tokens_to_ids('\n'))

32003
0


In [75]:
# print(input_id.shape)
# print(len(tokenizer.batch_decode(input_id)))

for tok, decoding in zip(list(input_id), tokenizer.batch_decode(input_id)):
    print(str(tok.item()) + '\t' +  repr(decoding))

1	'<s>'
32003	'<|im_start|>'
1587	'system'
13	'\n'
1976	'You'
460	'are'
264	'a'
10865	'helpful'
28725	','
3116	'respect'
1007	'ful'
304	'and'
6858	'honest'
13892	'assistant'
28723	'.'
17484	'Always'
4372	'answer'
390	'as'
1316	'help'
3071	'fully'
390	'as'
2572	'possible'
28723	'.'
32002	'<|im_end|>'
28705	''
13	'\n'
32003	'<|im_start|>'
2188	'user'
13	'\n'
32000	'<image>'
28705	''
13	'\n'
4957	'Do'
368	'you'
1032	'see'
707	'any'
1579	'bus'
274	'es'
442	'or'
8300	'cars'
28804	'?'
13	'\n'
2820	'An'
16981	'swer'
272	'the'
2996	'question'
1413	'using'
264	'a'
2692	'single'
1707	'word'
442	'or'
14804	'phrase'
28723	'.'
32002	'<|im_end|>'
28705	''
13	'\n'
32003	'<|im_start|>'
13892	'assistant'
13	'\n'
5613	'Yes'
32002	'<|im_end|>'
28705	''
13	'\n'
32003	'<|im_start|>'
2188	'user'
13	'\n'
2301	'Is'
736	'there'
707	'any'
17210	'fence'
297	'in'
456	'this'
9180	'photograph'
28804	'?'
32002	'<|im_end|>'
28705	''
13	'\n'
32003	'<|im_start|>'
13892	'assistant'
13	'\n'
5613	'Yes'
32002	'<|im_end|>'


In [83]:
sep_token = tokenizer.convert_tokens_to_ids('<|im_end|>')
print(sep_token)

32002


AttributeError: 'list' object has no attribute 'split'

In [84]:
# trying to split based off of TOKENIZED output

# TODO maybe try to build targets (list) based off of appending and switching mode when we see a certain thing :|
# but wanna do chunks probably

IMAGE_TOKEN_INDEX = tokenizer.convert_tokens_to_ids('<image>') # i think
_HF_IGNORE_INDEX = -100
sep_token = tokenizer.convert_tokens_to_ids('<|im_end|>')
# start = '<|im_start|>'
# sep_n_tokens = 2 # NOTE: check if '<|im_end|>\n' splits into two tokens bc will need that for later

rounds = input_id.split(sep_token) 
cur_len = 0 #  1 ??
# target[:cur_len] = _HF_IGNORE_INDEX

target = input_id.clone() # copy.deepcopy(input_id)
print('targets before', target)
for i, rou in enumerate(rounds):
    print(rou)

    # cur_len += round_len
    # print('cur len', cur_len)

    print('--')
    # break
print('targets after', target)

targets before tensor([    1, 32003,  1587,    13,  1976,   460,   264, 10865, 28725,  3116,
         1007,   304,  6858, 13892, 28723, 17484,  4372,   390,  1316,  3071,
          390,  2572, 28723, 32002, 28705,    13, 32003,  2188,    13, 32000,
        28705,    13,  4957,   368,  1032,   707,  1579,   274,   442,  8300,
        28804,    13,  2820, 16981,   272,  2996,  1413,   264,  2692,  1707,
          442, 14804, 28723, 32002, 28705,    13, 32003, 13892,    13,  5613,
        32002, 28705,    13, 32003,  2188,    13,  2301,   736,   707, 17210,
          297,   456,  9180, 28804, 32002, 28705,    13, 32003, 13892,    13,
         5613, 32002, 28705,    13, 32003,  2188,    13,  4957,   368,  1032,
          264,  1579,   369,   349,   459,  2760, 28804, 32002, 28705,    13,
        32003, 13892,    13,  2501, 32002, 28705,    13, 32003,  2188,    13,
        26703,  3181,   349,   369,  1579, 28804, 32002, 28705,    13, 32003,
        13892,    13,  7516, 32002, 28705,    13,

In [ ]:
# add end signal and concatenate together
conversations = []
for source in sources:
    assert len(source) == 2
    assert DEFAULT_IMAGE_TOKEN in source[0]['value']
    source[0]['value'] = DEFAULT_IMAGE_TOKEN
    conversation = source[0]['value'] + source[1]['value'] + conversation_lib.default_conversation.sep
    conversations.append(conversation)
# tokenize conversations
input_ids = [tokenizer_image_token(prompt, tokenizer, return_tensors='pt') for prompt in conversations]
targets = copy.deepcopy(input_ids)
for target, source in zip(targets, sources):
    tokenized_len = len(tokenizer_image_token(source[0]['value'], tokenizer))
    target[:tokenized_len] = IGNORE_INDEX

return dict(input_ids=input_ids, labels=targets)

In [10]:
tokenizer_1 = AutoTokenizer.from_pretrained('lmsys/vicuna-7b-v1.5')
vicuna_chat_template = "{% if messages[0]['role'] == 'system' %} {% set loop_messages = messages[1:] %} {% set system_message = messages[0]['content'].strip() + '\n\n' %} {% else %} {% set loop_messages = messages %} {% set system_message = '' %} {% endif %} {{ bos_token + system_message }} {% for message in loop_messages %} {% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %} {{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }} {% endif %} {% if message['role'] == 'user' %} {{ 'USER: ' + message['content'].strip() + '\n' }} {% elif message['role'] == 'assistant' %} {{ 'ASSISTANT: ' + message['content'].strip() + eos_token + '\n' }} {% endif %} {% endfor %} {% if add_generation_prompt %} {{ 'ASSISTANT:' }} {% endif %}"

In [12]:
if tokenizer_1.chat_template:
    print('hi')

hi


In [11]:
tokenizer_1.chat_template = vicuna_chat_template

In [37]:
chat = [
   {"role": "user", "content": "Hello, how are you?"},
   {"role": "assistant", "content": "I'm doing great. How can I help you today?"},
   {"role": "user", "content": "I'd like to show off how chat templating works!"},
]

out = tokenizer_1.apply_chat_template(chat, tokenize=False)
print(out)

    <s>    USER: Hello, how are you?
     ASSISTANT: I'm doing great. How can I help you today?</s>
     USER: I'd like to show off how chat templating works!
   


In [20]:
url = "https://us.feliway.com/cdn/shop/articles/10_fascinating_facts_about_black_cats-3.jpg?v=1667409596&width=19201x//us.feliway.com/cdn/shop/articles/10_fascinating_facts_about_black_cats-3.jpg?v=1667409596&width=38402x"
# url = "https://www.ilankelman.org/stopsigns/australia.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [2]:
processor = AutoProcessor.from_pretrained("llava-hf/bakLlava-v1-hf")

preprocessor_config.json: 100%|██████████| 505/505 [00:00<00:00, 3.21MB/s]
tokenizer_config.json: 100%|██████████| 1.41k/1.41k [00:00<00:00, 11.1MB/s]
tokenizer.model: 100%|██████████| 493k/493k [00:00<00:00, 20.9MB/s]
tokenizer.json: 100%|██████████| 1.80M/1.80M [00:00<00:00, 20.9MB/s]
added_tokens.json: 100%|██████████| 41.0/41.0 [00:00<00:00, 362kB/s]
special_tokens_map.json: 100%|██████████| 552/552 [00:00<00:00, 4.95MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [23]:
inputs = processor(text='blah blah', images=None, return_tensors="pt")
print(inputs)

{'input_ids': tensor([[  1, 843, 912, 843, 912]]), 'attention_mask': tensor([[1, 1, 1, 1, 1]]), 'pixel_values': None}


In [5]:
model_input_ids = torch.tensor([[    1,   523, 28766,   321, 28730,  2521, 28766, 28767,  1587,    13,                                                                                           
          1976,   460,   264, 10865, 28725,  3116,  1007,   304,  6858, 13892,                                                                                                     
         28723, 17484,  4372,   390,  1316,  3071,   390,  2572, 26364, 28766,                                                                                                     
           321, 28730,   416, 28766, 28767, 28705,    13, 28789, 28766,   321,                                                                                                     
         28730,  2521, 28766, 28767,  2188,    13, 32000,   259,    13,  3195,                                                                                                     
         28742, 28713,   272,  3036,   302,   272,  3469, 28804, 28789, 28766,                                                                                                     
           321, 28730,   416, 28766, 28767, 28705,    13, 28789, 28766,   321,                                                                                                     
         28730,  2521, 28766, 28767, 13892,    13]])

In [4]:
processor.tokenizer

LlamaTokenizerFast(name_or_path='llava-hf/bakLlava-v1-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [18]:
res = processor.batch_decode(model_input_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
print(repr(res))
print(repr(res.replace(' ', '-')))

"<s> <|im_start|> system\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|> \n<|im_start|> user\n<image>  \nWhat's the content of the image?<|im_end|> \n<|im_start|> assistant\n"
"<s>-<|im_start|>-system\nYou-are-a-helpful,-respectful-and-honest-assistant.-Always-answer-as-helpfully-as-possible.<|im_end|>-\n<|im_start|>-user\n<image>--\nWhat's-the-content-of-the-image?<|im_end|>-\n<|im_start|>-assistant\n"


In [19]:
print(repr(res.replace(' ', '-')))
other_model_input_ids = torch.tensor([[    1,   523, 28766,   321, 28730,  2521, 28766, 28767,  1587,    13,                                                                                           
          1976,   460,   264, 10865, 28725,  3116,  1007,   304,  6858, 13892,                                                                                                     
         28723, 17484,  4372,   390,  1316,  3071,   390,  2572, 26364, 28766,                                                                                                     
           321, 28730,   416, 28766, 28767, 28705,    13, 28789, 28766,   321,                                                                                                     
         28730,  2521, 28766, 28767,  2188,    13, 32000,   260,    13,  3195,                                                                                                     
          6183,  4148,  2496,   349,   356,   272,   287,  7041,   297,   272,                                                                                                     
          5414, 28804,    13,  2820, 16981,   272,  2996,  1413,   264,  2692,                                                                                                     
          1707,   442, 14804, 26364, 28766,   321, 28730,   416, 28766, 28767,                                                                                                     
         28705,    13, 28789, 28766,   321, 28730,  2521, 28766, 28767, 13892,                                                                                                     
            13]])

res = processor.batch_decode(other_model_input_ids, skip_special_tokens=False, clean_up_tokenization_spaces=False)[0]
print(repr(res.replace(' ', '-')))

"<s>-<|im_start|>-system\nYou-are-a-helpful,-respectful-and-honest-assistant.-Always-answer-as-helpfully-as-possible.<|im_end|>-\n<|im_start|>-user\n<image>--\nWhat's-the-content-of-the-image?<|im_end|>-\n<|im_start|>-assistant\n"
'<s>-<|im_start|>-system\nYou-are-a-helpful,-respectful-and-honest-assistant.-Always-answer-as-helpfully-as-possible.<|im_end|>-\n<|im_start|>-user\n<image>----\nWhat-credit-card-company-is-on-the-banner-in-the-background?\nAnswer-the-question-using-a-single-word-or-phrase.<|im_end|>-\n<|im_start|>-assistant\n'


In [2]:
from transformers import (AutoConfig, PreTrainedTokenizerBase, AutoModel,
                          LlavaForConditionalGeneration, LlavaConfig,
                          CLIPVisionConfig, MistralConfig, LlamaConfig, AutoTokenizer)

tokenizer_1 = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')

tokenizer_2 = AutoTokenizer.from_pretrained('llava-hf/bakLlava-v1-hf')

tokenizer_1_add_tokens = AutoTokenizer.from_pretrained('mistralai/Mistral-7B-v0.1')
tokenizer_1_add_tokens.add_tokens(['<image>', '<pad>'], special_tokens=True)

tokenizer_1.chat_template = "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}"


/usr/lib/python3/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
print(tokenizer_1)

tokenizer_1.add_tokens(['<image>', '<pad>'], special_tokens=True)

print(tokenizer_1)

tokenizer_1.add_tokens(['<image>', '<pad>'], special_tokens=True)

print(tokenizer_1)

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstri

In [18]:
! pip install tiktoken
tok = AutoTokenizer.from_pretrained('rajammanabrolu/gpt-4-chat', trust_remote_code=True)
print(tok)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 50.6 MB/s eta 0:00:0000:01


added_tokens.json: 100%|██████████| 74.0/74.0 [00:00<00:00, 612kB/s]
special_tokens_map.json: 100%|██████████| 200/200 [00:00<00:00, 1.39MB/s]


TiktokenTokenizerWrapper(name_or_path='rajammanabrolu/gpt-4-chat', vocab_size=100277, model_max_length=8192, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|pad|>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>']}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	100257: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100277: AddedToken("<|pad|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100278: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100279: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}


In [22]:
tokenizer_1

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [26]:
tokenizer_1.add_special_tokens({'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>'}) # do we want to do this???

tokenizer_1

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [27]:
len(sample['conversations'])

6

In [33]:
SYSTEM = 'You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.'

formatted_chat_dicts = []
for i in range(len(sample['conversations']) // 2): # make a formatted convo for each turn
    # print((i+1)*2)
    # print(sample['conversations'][:(i+1)*2])
    # print()
    formatted_convo = [{'role': 'system', 'content': SYSTEM}]
    for line in sample['conversations'][:(i+1)*2]:
        if line['from'] == 'human':
            formatted_convo.append({'role': 'user', 'content': line['value']})
        elif line['from'] == 'gpt':
            formatted_convo.append({'role': 'assistant', 'content': line['value']})
    formatted_chat_dicts.append({'messages': formatted_convo})


print('original conversation:')
for line in sample['conversations']:
    print(line)

# chat_formatted_dict = {'messages': formatted_convos}
# print(chat_formatted_dict)

print(len(formatted_chat_dicts), 'formatted convos')

original conversation:
{'from': 'human', 'value': '<image>\nWhat are the colors of the bus in the image?'}
{'from': 'gpt', 'value': 'The bus in the image is white and red.'}
{'from': 'human', 'value': 'What feature can be seen on the back of the bus?'}
{'from': 'gpt', 'value': 'The back of the bus features an advertisement.'}
{'from': 'human', 'value': 'Is the bus driving down the street or pulled off to the side?'}
{'from': 'gpt', 'value': 'The bus is driving down the street, which is crowded with people and other vehicles.'}
3 formatted convos


In [37]:
from llmfoundry.data.finetuning.tasks import _slice_chat_formatted_example
prompt, response = _slice_chat_formatted_example(formatted_chat_dicts[2], tokenizer_1)

print('PROMPT:', prompt)
print('RESPONSE:', response)

PROMPT: <|im_start|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|>
<|im_start|>user
<image>
What are the colors of the bus in the image?<|im_end|>
<|im_start|>assistant
The bus in the image is white and red.<|im_end|>
<|im_start|>user
What feature can be seen on the back of the bus?<|im_end|>
<|im_start|>assistant
The back of the bus features an advertisement.<|im_end|>
<|im_start|>user
Is the bus driving down the street or pulled off to the side?<|im_end|>

RESPONSE: <|im_start|>assistant
The bus is driving down the street, which is crowded with people and other vehicles.<|im_end|>



PROMPT <|im_start|>system
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|>
<|im_start|>user
<image>
What are the colors of the bus in the image?<|im_end|>
<|im_start|>assistant
The bus in the image is white and red.<|im_end|>
<|im_start|>user
What feature can be seen on the back of the bus?<|im_end|>
<|im_start|>assistant
The back of the bus features an advertisement.<|im_end|>
<|im_start|>user
Is the bus driving down the street or pulled off to the side?<|im_end|>

RESPONSE: <|im_start|>assistant
The bus is driving down the street, which is crowded with people and other vehicles.<|im_end|>



In [4]:
tokenizer_2

LlamaTokenizerFast(name_or_path='llava-hf/bakLlava-v1-hf', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [8]:
tokenizer_1_add_tokens

LlamaTokenizerFast(name_or_path='mistralai/Mistral-7B-v0.1', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32000: AddedToken("<image>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

: 

In [4]:
vision_tower_2

CLIPVisionModel(
  (vision_model): CLIPVisionTransformer(
    (embeddings): CLIPVisionEmbeddings(
      (patch_embedding): Conv2d(3, 1024, kernel_size=(14, 14), stride=(14, 14), bias=False)
      (position_embedding): Embedding(577, 1024)
    )
    (pre_layrnorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-23): 24 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (layer_norm1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=1024, out_features=4096, bias=

In [13]:
from transformers import AutoConfig, CLIPVisionConfig

vision_tower = 'openai/clip-vit-large-patch14-336'
vision_config = CLIPVisionConfig.from_pretrained(vision_tower)

print(vision_config)



model_name_or_path = 'mistralai/Mistral-7B-v0.1'
text_config = AutoConfig.from_pretrained(model_name_or_path)

print(text_config)

CLIPVisionConfig {
  "attention_dropout": 0.0,
  "dropout": 0.0,
  "hidden_act": "quick_gelu",
  "hidden_size": 1024,
  "image_size": 336,
  "initializer_factor": 1.0,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "model_type": "clip_vision_model",
  "num_attention_heads": 16,
  "num_channels": 3,
  "num_hidden_layers": 24,
  "patch_size": 14,
  "projection_dim": 768,
  "transformers_version": "4.37.2"
}

MistralConfig {
  "_name_or_path": "mistralai/Mistral-7B-v0.1",
  "architectures": [
    "MistralForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 32768,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-05,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "t

In [12]:
vision_config

CLIPConfig {
  "_name_or_path": "openai/clip-vit-large-patch14-336",
  "architectures": [
    "CLIPModel"
  ],
  "initializer_factor": 1.0,
  "logit_scale_init_value": 2.6592,
  "model_type": "clip",
  "projection_dim": 768,
  "text_config": {
    "dropout": 0.0,
    "hidden_size": 768,
    "intermediate_size": 3072,
    "model_type": "clip_text_model",
    "num_attention_heads": 12,
    "projection_dim": 768
  },
  "torch_dtype": "float32",
  "transformers_version": "4.37.2",
  "vision_config": {
    "dropout": 0.0,
    "hidden_size": 1024,
    "image_size": 336,
    "intermediate_size": 4096,
    "model_type": "clip_vision_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 24,
    "patch_size": 14,
    "projection_dim": 768
  }
}

In [14]:
llava_config = LlavaConfig(vision_config, text_config)
print(llava_config)

LlavaConfig {
  "ignore_index": -100,
  "image_token_index": 32000,
  "model_type": "llava",
  "projector_hidden_act": "gelu",
  "text_config": {
    "_name_or_path": "mistralai/Mistral-7B-v0.1",
    "architectures": [
      "MistralForCausalLM"
    ],
    "intermediate_size": 14336,
    "max_position_embeddings": 32768,
    "model_type": "mistral",
    "num_key_value_heads": 8,
    "rms_norm_eps": 1e-05,
    "sliding_window": 4096,
    "torch_dtype": "bfloat16"
  },
  "transformers_version": "4.37.2",
  "vision_config": {
    "dropout": 0.0,
    "hidden_size": 1024,
    "image_size": 336,
    "intermediate_size": 4096,
    "model_type": "clip_vision_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 24,
    "patch_size": 14,
    "projection_dim": 768
  },
  "vision_feature_layer": -2,
  "vision_feature_select_strategy": "default",
  "vocab_size": 32000
}



In [20]:
llava_config # .text_config._name_or_path

LlavaConfig {
  "ignore_index": -100,
  "image_token_index": 32000,
  "model_type": "llava",
  "projector_hidden_act": "gelu",
  "text_config": {
    "_name_or_path": "mistralai/Mistral-7B-v0.1",
    "architectures": [
      "MistralForCausalLM"
    ],
    "intermediate_size": 14336,
    "max_position_embeddings": 32768,
    "model_type": "mistral",
    "num_key_value_heads": 8,
    "rms_norm_eps": 1e-05,
    "sliding_window": 4096,
    "torch_dtype": "bfloat16"
  },
  "transformers_version": "4.37.2",
  "vision_config": {
    "dropout": 0.0,
    "hidden_size": 1024,
    "image_size": 336,
    "intermediate_size": 4096,
    "model_type": "clip_vision_model",
    "num_attention_heads": 16,
    "num_hidden_layers": 24,
    "patch_size": 14,
    "projection_dim": 768
  },
  "vision_feature_layer": -2,
  "vision_feature_select_strategy": "default",
  "vocab_size": 32000
}

In [2]:
yaml_path = '/mnt/workdisk/jasmine/yamls/llava/mpt-7b-llava-local.yaml'
cfg = om.load(yaml_path)

validate_config(cfg)
om.resolve(cfg)
print(cfg)

# Mandatory model training configs
model_config: DictConfig = pop_config(cfg, 'model', must_exist=True)
tokenizer_config: Dict[str, Any] = pop_config(cfg,
                                                'tokenizer',
                                                must_exist=True,
                                                convert=True)
optimizer_config: Dict[str, Any] = pop_config(cfg,
                                                'optimizer',
                                                must_exist=True,
                                                convert=True)
scheduler_config: Dict[str, Any] = pop_config(cfg,
                                                'scheduler',
                                                must_exist=True,
                                                convert=True)
train_loader_config: DictConfig = pop_config(cfg,
                                                'train_loader',
                                                must_exist=True)
max_duration: Union[int, str] = pop_config(cfg,
                                            'max_duration',
                                            must_exist=True)
eval_interval: Union[int, str] = pop_config(cfg,
                                            'eval_interval',
                                            must_exist=True)
precision: str = pop_config(cfg, 'precision', must_exist=True)
max_seq_len: int = pop_config(cfg, 'max_seq_len', must_exist=True)
device_train_batch_size = 1

{'max_seq_len': 1024, 'global_seed': 17, 'model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'run_name': 'bakllava-cc3m-clip-llm-pretrained-refactor', 'model': {'name': 'hf_multimodal_causal_lm', 'pretrained_model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'llm_model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'vision_model_name_or_path': 'openai/clip-vit-large-patch14-336', 'pretrained': False, 'train_llm': False}, 'tokenizer': {'name': 'mistralai/Mistral-7B-v0.1', 'kwargs': {'model_max_length': 1024, 'chat_template': "{% if messages[0]['role'] == 'system' %}{% set loop_messages = messages[1:] %}{% set system_message = messages[0]['content'] %}{% elif USE_DEFAULT_PROMPT == true and not 'system' in messages[0]['role'] %}{% set loop_messages = messages %}{% set system_message = 'DEFAULT_SYSTEM_PROMPT' %}{% else %}{% set loop_messages = messages %}{% set system_message = false %}{% endif %}{% for message in loop_messages %}{% if loop.index0 == 0 %}{% if system_message != false %}{{ '<|im

In [3]:
model_config

{'name': 'hf_multimodal_causal_lm', 'pretrained_model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'llm_model_name_or_path': 'mistralai/Mistral-7B-v0.1', 'vision_model_name_or_path': 'openai/clip-vit-large-patch14-336', 'pretrained': False, 'train_llm': False}

In [3]:
# Build tokenizer
tokenizer_name = tokenizer_config['name']
tokenizer_kwargs = tokenizer_config.get('kwargs', {})
tokenizer = build_tokenizer(tokenizer_name, tokenizer_kwargs)

# Build train loader
streaming.base.util.clean_stale_shared_memory()
train_loader = build_dataloader(
    train_loader_config,
    tokenizer,
    device_train_batch_size,
)


Because `predownload` was not specified, it will default to 8*batch_size if batch_size is not None, otherwise 64. Prior to Streaming v0.7.0, `predownload` defaulted to max(batch_size, 256 * batch_size // num_canonical_nodes).


In [4]:
# Initialize model
model = build_composer_model(model_config, tokenizer)

if model_config.get('master_weights_dtype') in ('bf16', 'bfloat16'):
    model = model.to(dtype=torch.bfloat16)
elif model_config.get('master_weights_dtype') in ('f16', 'float16'):
    model = model.to(dtype=torch.float16)

# Log number of parameters
n_params = sum(p.numel() for p in model.parameters())
print('n params', n_params)

# Optimizer
optimizer_name: str = optimizer_config.pop('name')
optimizer = build_optimizer(model, optimizer_name, optimizer_config)

/usr/lib/python3/dist-packages/transformers/models/auto/configuration_auto.py:1085: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:11<00:00,  3.00s/it]


n params 7566743552


In [ ]:
# model.model.vision_tower.requires_grad_(False)
# model.model.language_model.requires_grad_(False)
# model.model.multi_modal_projector.requires_grad_(True) # explicit

In [ ]:
# for name, p in model.named_parameters():
#     print(name, 'requires grad', p.requires_grad)

In [5]:
batch = torch.load('batch.ckpt')

for key in batch:
    print(key, batch[key].shape)

input_ids torch.Size([1, 512])
attention_mask torch.Size([1, 512])
labels torch.Size([1, 512])
llava_labels torch.Size([1, 86])
bidirectional_mask torch.Size([1, 512])
pixel_values torch.Size([1, 3, 336, 336])


In [24]:
from composer.metrics.nlp import LanguageCrossEntropy
from torch import nn
_HF_IGNORE_INDEX = -100

composer_xent = LanguageCrossEntropy(ignore_index=_HF_IGNORE_INDEX)

torch_xent = nn.CrossEntropyLoss()

In [62]:
from torch import nn

model = model.half()
model = model.to('cuda')


for i, key in enumerate(batch):
    batch[key] = batch[key].to('cuda')
    if i==2:
        break

out, aug_label = model(batch)

OutOfMemoryError: CUDA out of memory. Tried to allocate 146.00 MiB. GPU 0 has a total capacty of 39.39 GiB of which 109.94 MiB is free. Process 1180216 has 39.27 GiB memory in use. Of the allocated memory 38.06 GiB is allocated by PyTorch, and 720.13 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [28]:
batch['attention_mask']

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0

In [54]:
print(out.loss)
composer_xent.update(out.logits.cpu()[:,256:,:], batch['labels'].cpu()[:,256:])
print(composer_xent.compute())

# shifting gives us 17.7 --> 0.52
# compare all but last logit to labels starting at 1
print(torch_xent(out.logits.cpu()[0,:-1,:], batch['labels'].cpu()[0,1:]))

tensor(0.5281, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(17.7352)
tensor(0.5281, grad_fn=<NllLossBackward0>)


In [40]:
shift_attention_mask = batch['attention_mask'][..., 1:]
logits = out.logits
labels = batch['labels']
shift_logits = logits[..., :-1, :][shift_attention_mask.to(logits.device) != 0].contiguous()
shift_labels = labels[..., 1:][shift_attention_mask.to(labels.device) != 0].contiguous()

print(shift_logits.shape)
print(shift_labels.shape)

print(torch_xent(shift_logits.cpu(), shift_labels.cpu()))

torch.Size([166, 32064])
torch.Size([166])
tensor(0.5281, grad_fn=<NllLossBackward0>)


In [58]:
# print(torch.argmax(shift_logits,dim=-1)[100:])
# print(shift_labels[100:])

print(torch.argmax(out.logits,dim=-1)[:,-10:])
print(batch['labels'][:,-10:])

tensor([[  263, 28742, 28713,  4501, 28723,   272,  3469, 28723,     2,     2]],
       device='cuda:0')
tensor([[ 2204,   263, 28742, 28713,  4501,   297,   272,  3469, 28723,     2]],
       device='cuda:0')


In [50]:
labels = shift_labels[75:] # batch['labels'] # shift_labels[75:] # 
labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)

['',
 '',
 '',
 '',
 '',
 'The',
 'unique',
 'characteristic',
 'visible',
 'on',
 'the',
 'l',
 'amb',
 's',
 "'",
 'wool',
 'is',
 'the',
 'blue',
 'paint',
 'mark',
 'ings',
 '.',
 'These',
 'mark',
 'ings',
 'are',
 'often',
 'used',
 'by',
 'farmers',
 'to',
 'distinguish',
 'and',
 'identify',
 'the',
 'sheep',
 'or',
 'to',
 'indicate',
 'certain',
 'information',
 'about',
 'them',
 ',',
 'such',
 'as',
 'ownership',
 'or',
 'management',
 'details',
 '.',
 'The',
 'presence',
 'of',
 'these',
 'blue',
 'paint',
 'mark',
 'ings',
 'on',
 'the',
 'young',
 'sheep',
 'lying',
 'next',
 'to',
 'the',
 'fence',
 'adds',
 'an',
 'unusual',
 'aspect',
 'to',
 'their',
 'appearance',
 ',',
 'which',
 'might',
 'catch',
 'the',
 'view',
 'er',
 "'",
 's',
 'attention',
 'in',
 'the',
 'image',
 '.',
 '']

In [52]:
labels = torch.argmax(shift_logits,dim=-1)[65:] # batch['labels'] # shift_labels[75:] # 
labels = torch.where(labels != -100, labels, tokenizer.pad_token_id)
tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)

['start',
 '|',
 '>',
 'blue',
 'blue',
 '|',
 'im',
 '_',
 'start',
 '|',
 '>',
 'system',
 'istant',
 ':',
 'blue',
 'Q',
 'blue',
 'characteristic',
 'visible',
 'on',
 'the',
 'l',
 'amb',
 's',
 "'",
 'wool',
 'is',
 'the',
 'blue',
 'paint',
 'mark',
 'ings',
 '.',
 'These',
 'mark',
 'ings',
 'are',
 'often',
 'used',
 'by',
 'farmers',
 'to',
 'identify',
 'and',
 'identify',
 'the',
 'sheep',
 ',',
 'to',
 'indicate',
 'certain',
 'information',
 'about',
 'them',
 ',',
 'such',
 'as',
 'ownership',
 'or',
 'management',
 'details',
 '.',
 'The',
 'blue',
 'of',
 'these',
 'blue',
 'paint',
 'mark',
 'ings',
 'on',
 'the',
 'l',
 'sheep',
 'lying',
 'next',
 'to',
 'the',
 'fence',
 'adds',
 'an',
 'unusual',
 'aspect',
 'to',
 'their',
 'appearance',
 ',',
 'as',
 'might',
 'catch',
 'the',
 'view',
 'er',
 "'",
 's',
 'attention',
 '.',
 'the',
 'image',
 '.',
 '']

In [10]:
labels = torch.where(aug_label != -100, aug_label, tokenizer.pad_token_id)
tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False)

["The unique characteristic visible on the lambs' wool is the blue paint markings. These markings are often used by farmers to distinguish and identify the sheep or to indicate certain information about them, such as ownership or management details. The presence of these blue paint markings on the young sheep lying next to the fence adds an unusual aspect to their appearance, which might catch the viewer's attention in the image."]

In [67]:
for i, key in enumerate(batch):

    labels = torch.where(batch['input_ids'] != -100, batch['input_ids'], tokenizer.pad_token_id)
    print(tokenizer.batch_decode(labels, skip_special_tokens=True, clean_up_tokenization_spaces=False))

    if i==2:
        break

["<|im_start|>system\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|>\n<|im_start|>user\n \nWhat unique characteristic is visible on the lambs' wool?<|im_end|>\n<|im_start|>assistant\n The unique characteristic visible on the lambs' wool is the blue paint markings. These markings are often used by farmers to distinguish and identify the sheep or to indicate certain information about them, such as ownership or management details. The presence of these blue paint markings on the young sheep lying next to the fence adds an unusual aspect to their appearance, which might catch the viewer's attention in the image."]
["<|im_start|>system\nYou are a helpful, respectful and honest assistant. Always answer as helpfully as possible.<|im_end|>\n<|im_start|>user\n \nWhat unique characteristic is visible on the lambs' wool?<|im_end|>\n<|im_start|>assistant\n The unique characteristic visible on the lambs' wool is the blue paint markings. These m

: 

In [66]:
len(batch)

6

In [ ]:
optimizer.zero_grad()

out.loss.backward()

optimizer.step()

In [ ]:
print(new_labels.shape)

import matplotlib.pyplot as plt
plt.plot(new_labels.cpu().numpy()[0])#[999:])
plt.show()

new_labels[0][998:]
print(batch['labels'].shape)

plt.plot(batch['labels'].cpu().numpy()[0][425:])
plt.show()

plt.plot(out['logits'].argmax(dim=-1).cpu().numpy()[0][-max_seq_len:])
plt.show()

plt.plot(batch['labels'].cpu().numpy()[0])
plt.show()

In [ ]:
for key in out:
    print(key)
print(out['loss'])
print('logits shape', out['logits'].shape)

In [ ]:
print()

In [ ]:
for key in batch:
    print(key, batch[key].type())